### 230808

In [84]:
# from docx import Document
# from docx.shared import Inches
import docx 
from docx.shared import Inches

from docx.oxml import OxmlElement
from docx.oxml.ns import qn

import subprocess

In [27]:
write_path = '/home/k/DATA/CV/'


In [92]:


def set_cell_border(cell, **kwargs):
    """
    Set cell`s border
    Usage:

    set_cell_border(
        cell,
        top={"sz": 12, "val": "single", "color": "#FF0000", "space": "0"},
        bottom={"sz": 12, "color": "#00FF00", "val": "single"},
        start={"sz": 24, "val": "dashed", "shadow": "true"},
        end={"sz": 12, "val": "dashed"},
    )
    """
    tc = cell._tc
    tcPr = tc.get_or_add_tcPr()

    # check for tag existnace, if none found, then create one
    tcBorders = tcPr.first_child_found_in("w:tcBorders")
    if tcBorders is None:
        tcBorders = OxmlElement('w:tcBorders')
        tcPr.append(tcBorders)

    # list over all available tags
    for edge in ('start', 'top', 'end', 'bottom', 'insideH', 'insideV'):
        edge_data = kwargs.get(edge)
        if edge_data:
            tag = 'w:{}'.format(edge)

            # check for tag existnace, if none found, then create one
            element = tcBorders.find(qn(tag))
            if element is None:
                element = OxmlElement(tag)
                tcBorders.append(element)

            # looks like order of attributes is important
            for key in ["sz", "val", "color", "space", "shadow"]:
                if key in edge_data:
                    element.set(qn('w:{}'.format(key)), str(edge_data[key]))

# def modifyBorder(table):
#     tbl = table._tbl # get xml element in table
#     for cell in tbl.iter_tcs():
#         tcPr = cell.tcPr # get tcPr element, in which we can define style of borders
#         tcBorders = OxmlElement('w:tcBorders')
#         top = OxmlElement('w:top')
#         top.set(qn('w:val'), 'nil')
        
#         left = OxmlElement('w:left')
#         left.set(qn('w:val'), 'nil')
        
#         bottom = OxmlElement('w:bottom')
#         bottom.set(qn('w:val'), 'nil')
#         bottom.set(qn('w:sz'), '4')
#         bottom.set(qn('w:space'), '0')
#         bottom.set(qn('w:color'), 'auto')

#         right = OxmlElement('w:right')
#         right.set(qn('w:val'), 'nil')

#         tcBorders.append(top)
#         tcBorders.append(left)
#         tcBorders.append(bottom)
#         tcBorders.append(right)
#         tcPr.append(tcBorders)

In [129]:
def indent_table(table, indent):
    '''Where indent is in twips, 
    e.g. 1cm is about 569
    1 inch is 1440'''
    # noinspection PyProtectedMember
    tbl_pr = table._element.xpath('w:tblPr')
    if tbl_pr:
        e = OxmlElement('w:tblInd')
        e.set(qn('w:w'), str(indent))
        e.set(qn('w:type'), 'dxa')
        tbl_pr[0].append(e)

In [136]:
# Create a document
doc = docx.Document()

# General Style
style = doc.styles['Normal']
font = style.font
font.name = 'Arial'
font.size = docx.shared.Pt(11)

# Add a paragraph to the document
p = doc.add_paragraph()

# Add some formatting to the paragraph
p.paragraph_format.line_spacing = 1
p.paragraph_format.space_after = docx.shared.Pt(10)

# Add a run to the paragraph
run = p.add_run("Keiland W. Cooper")
run.bold = True
run.font.name = 'Arial'
run.font.size = docx.shared.Pt(18)

# Add another paragraph (left blank for an empty line)
# doc.add_paragraph()

# Add heading
p = doc.add_paragraph()
run = p.add_run("Education")
run.bold = True
run.font.name = 'Arial'
run.font.size = docx.shared.Pt(14)

# P2
p = doc.add_paragraph()
p.style = doc.styles['Normal']
p.paragraph_format.line_spacing = 1
p.paragraph_format.space_after = docx.shared.Pt(10)
p.paragraph_format.left_indent = Inches(1)


# Add a table
n_rows = 4
table = doc.add_table(rows=n_rows, cols=3)
for row in table.rows:
    row_cells = row.cells
    row_cells[0].text = 'University of X'
    row_cells[1].text = str(2 + 2)
    row_cells[2].text = str(2022)
    

# Style the table
for row in table.rows:
    for cell in row.cells:
            set_cell_border(
                cell,
                top={"sz": 1, "val": "single", "color": "#FFFFFF"},
                bottom={"sz": 1, "val": "single", "color": "#FFFFFF"},
                start={"sz": 1, "val": "single", "color": "#FFFFFF"},
                end={"sz": 1, "val": "single", "color": "#FFFFFF"},
            )
indent_table(table, 1440)


run = p.add_run("University of California, Irvine")
run.add_break()
run = p.add_run("PHD")
run.add_break()
run = p.add_run("Norbert Fortin")

# Format the run
run.bold = False
run.font.name = 'Arial'
run.font.size = docx.shared.Pt(11)

# Add another paragraph (left blank for an empty line)
doc.add_paragraph()

# Add heading
p = doc.add_paragraph()
run = p.add_run("Publications")
run.bold = True
run.font.name = 'Arial'
run.font.size = docx.shared.Pt(14)



# Add another paragraph
p = doc.add_paragraph()

# Add a run and format it
run = p.add_run("This is my first python-docx tutorial!")
run.font.name = 'Arial'
run.font.size = docx.shared.Pt(12)

# doc.add_heading('Heading, level 1', level=1)
# doc.add_paragraph('Intense quote', style='Intense Quote')

doc.add_paragraph(
    'first item in unordered list', style='List Bullet'
)

# Add a table
n_rows = 4
table = doc.add_table(rows=1, cols=3)
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Qty'
hdr_cells[1].text = 'Id'
hdr_cells[2].text = 'Desc'
for row_i in range(n_rows):
    row_cells = table.add_row().cells
    row_cells[0].text = str(1 + row_i)
    row_cells[1].text = str(2 + row_i)
    row_cells[2].text = str(3 + row_i)

    # Style the table
    for cell in row_cells:
            set_cell_border(
                cell,
                top={"sz": 1, "val": "single", "color": "#FFFFFF"},
                bottom={"sz": 1, "val": "single", "color": "#FFFFFF"},
                start={"sz": 1, "val": "single", "color": "#FFFFFF"},
                end={"sz": 1, "val": "single", "color": "#FFFFFF"},
            )


In [137]:
# Save the document
doc.save(write_path+"docx-demo.docx")

In [138]:
subprocess.run(f'libreoffice  ~/DATA/CV/docx-demo.docx', shell=True)

CompletedProcess(args='libreoffice  ~/DATA/CV/docx-demo.docx', returncode=0)

In [ ]:
# Create a document
doc = docx.Document()

# Add a paragraph to the document
p = doc.add_paragraph()

# Add some formatting to the paragraph
p.paragraph_format.line_spacing = 1
p.paragraph_format.space_after = 0

# Add a run to the paragraph
run = p.add_run("Keiland W. Cooper")
run.bold = True
run.italic = True
run.font.name = 'Arial'
run.font.size = docx.shared.Pt(16)

# Add more text to the same paragraph
run = p.add_run(" Tutorial")

# Format the run
run.bold = True
run.font.name = 'Arial'
run.font.size = docx.shared.Pt(16)

# Add another paragraph (left blank for an empty line)
doc.add_paragraph()

# Add another paragraph
p = doc.add_paragraph()

# Add a run and format it
run = p.add_run("This is my first python-docx tutorial!")
run.font.name = 'Arial'
run.font.size = docx.shared.Pt(12)

# doc.add_heading('Heading, level 1', level=1)
# doc.add_paragraph('Intense quote', style='Intense Quote')

doc.add_paragraph(
    'first item in unordered list', style='List Bullet'
)

# Add a table
n_rows = 4
table = doc.add_table(rows=1, cols=3)
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Qty'
hdr_cells[1].text = 'Id'
hdr_cells[2].text = 'Desc'
for row_i in range(n_rows):
    row_cells = table.add_row().cells
    row_cells[0].text = str(1 + row_i)
    row_cells[1].text = str(2 + row_i)
    row_cells[2].text = str(3 + row_i)